In [4]:
import pandas as pd 
from sklearn.feature_extraction.text import CountVectorizer
from transformers import AutoTokenizer, AutoModel
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.dummy import DummyClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
from sklearn.metrics import balanced_accuracy_score, f1_score
from sklearn.base import clone
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import numpy as np
import torch
import gc
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.optimizers.experimental import AdamW
from tensorflow.keras.layers import Dense,GRU, Dropout, LayerNormalization, Concatenate, Conv1D, Flatten,Reshape
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import clone_model
from keras_nlp.layers import TransformerEncoder

from tqdm import tqdm

In [3]:
tokEmbs = tf.keras.Input(shape=(256,768))
docEmbs = tf.keras.Input(shape=(768,))

x = TransformerEncoder(intermediate_dim=512, num_heads=8, dropout=0.2)(tokEmbs)
x = TransformerEncoder(intermediate_dim=512, num_heads=8, dropout=0.2)(x)
x = TransformerEncoder(intermediate_dim=512, num_heads=8, dropout=0.2)(x)
x = GRU(units=512)(x)
x = LayerNormalization()(x)
x = Dense(256,activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(256,activation='relu')(x)
x = Reshape((1,-1))(x)

y = Dense(768,activation='relu')(docEmbs)
y = Dense(512,activation='relu')(docEmbs)
y = LayerNormalization()(y)
y = Dense(256,activation='relu')(y)
y = Dropout(0.2)(y)
y = Dense(256,activation='relu')(y)
y = Reshape((1,-1))(y)

z = Concatenate(axis=1)([y,x])
z = Conv1D(filters=128,kernel_size = 2)(z)
z = Flatten()(z)

z = Dense(128,activation='relu')(z)
z = Dropout(0.2)(z)
z = Dense(64,activation='relu')(z)
z = Dense(1,activation='sigmoid')(z)

MixModel = tf.keras.Model(inputs=[tokEmbs,docEmbs], outputs=z)

MixModel.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 256, 768)]   0           []                               
                                                                                                  
 transformer_encoder_3 (Transfo  (None, 256, 768)    3153152     ['input_3[0][0]']                
 rmerEncoder)                                                                                     
                                                                                                  
 transformer_encoder_4 (Transfo  (None, 256, 768)    3153152     ['transformer_encoder_3[0][0]']  
 rmerEncoder)                                                                                     
                                                                                            

In [5]:
tokEmbs = tf.keras.Input(shape=(256,768))
docEmbs = tf.keras.Input(shape=(768,))

x = TransformerEncoder(intermediate_dim=512, num_heads=8, dropout=0.2)(tokEmbs)
x = TransformerEncoder(intermediate_dim=512, num_heads=8, dropout=0.2)(x)
x = TransformerEncoder(intermediate_dim=512, num_heads=8, dropout=0.2)(x)
x = GRU(units=512)(x)
x = LayerNormalization()(x)
x = Dense(256,activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(256,activation='relu')(x)

x = Dense(128,activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(64,activation='relu')(x)
x = Dense(1,activation='sigmoid')(x)

TokModel = tf.keras.Model(inputs=[tokEmbs,docEmbs], outputs=x)

TokModel.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 256, 768)]   0           []                               
                                                                                                  
 transformer_encoder_6 (Transfo  (None, 256, 768)    3153152     ['input_5[0][0]']                
 rmerEncoder)                                                                                     
                                                                                                  
 transformer_encoder_7 (Transfo  (None, 256, 768)    3153152     ['transformer_encoder_6[0][0]']  
 rmerEncoder)                                                                                     
                                                                                            

In [6]:
tokEmbs = tf.keras.Input(shape=(256,768))
docEmbs = tf.keras.Input(shape=(768,))

y = Dense(768,activation='relu')(docEmbs)
y = Dense(512,activation='relu')(y)
y = LayerNormalization()(y)
y = Dense(256,activation='relu')(y)
y = Dropout(0.2)(y)
y = Dense(256,activation='relu')(y)

y = Dense(128,activation='relu')(y)
y = Dropout(0.2)(y)
y = Dense(64,activation='relu')(y)
y = Dense(1,activation='sigmoid')(y)

DocModel = tf.keras.Model(inputs=[tokEmbs,docEmbs], outputs=y)

DocModel.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_8 (InputLayer)           [(None, 768)]        0           []                               
                                                                                                  
 dense_23 (Dense)               (None, 768)          590592      ['input_8[0][0]']                
                                                                                                  
 dense_24 (Dense)               (None, 512)          393728      ['dense_23[0][0]']               
                                                                                                  
 layer_normalization_5 (LayerNo  (None, 512)         1024        ['dense_24[0][0]']               
 rmalization)                                                                               

In [7]:
labelled  =pd.read_csv('data/labels(1).csv')
labelled = labelled.sort_values('label').drop_duplicates('blockID')

In [8]:
tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-french-europeana-cased")
model =  AutoModel.from_pretrained('dbmdz/bert-base-french-europeana-cased')
model.to('cuda:1')
''

Some weights of the model checkpoint at dbmdz/bert-base-french-europeana-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


''

In [9]:
def getTokens(text):
    tokens = tokenizer.encode(text, add_special_tokens=False)
    if len(tokens)>256:
        while tokens[-1] !=3 and len(tokens)>256:
            tokens = tokens[:-1]
        while len(tokens)>256:
            tokens = tokens[:-2]
            tokens.append(3)
    elif len(tokens)<256:
        tokens+=[0]*(256-len(tokens))
    return tokens

def getEmbeds(tokens):
    tokEmbs = []
    docEmbs = []
    batchSize = 32
    attention_mask = torch.tensor([[1 if tok>0 else 0 for tok in toks] for toks in tokens]).type('torch.ShortTensor').to('cuda:1')
    tokens = torch.tensor(tokens).type('torch.IntTensor').to('cuda:1')
    with torch.no_grad():
        for batch in range(0,len(tokens),batchSize):

            output = model(input_ids=tokens[batch:batch+batchSize],
                            attention_mask=attention_mask[batch:batch+batchSize],
                            output_hidden_states=False,#True,
                            output_attentions=False)

            tEmbs = output.last_hidden_state#(sum(output.hidden_states[-4:])/4)
            tokEmbs.extend([tEmbs[ind,:,:].cpu().numpy().astype('float16')
                            for ind in range(len(tEmbs))])
            docEmbs.extend(list(output.pooler_output.cpu().numpy().astype('float16')))
            del output, tEmbs
            torch.cuda.empty_cache()
    return (docEmbs, tokEmbs,[mask for mask in attention_mask.cpu().numpy()])

def setBinaryLabel(label):
    if label.isnumeric():
        topic = int(label[1:-1]) if len(label) > 1 else 0
    else : 
        return label
    relevants = [1,2,4,6,8]
    return 1 if topic in relevants else 0

In [10]:
labelled['tokens'] = labelled['text'].apply(getTokens)

In [21]:
#model.to
labelled['docEmbs'], labelled['tokEmbs'],  labelled['mask'] = getEmbeds(np.array(labelled['tokens'].tolist()))

In [22]:
labelled.to_pickle('data/labelled.bz')

In [11]:
labelled = pd.read_pickle('data/labelled.bz')

In [12]:
labelled['label'] = labelled['label'].apply(setBinaryLabel)

In [13]:
labelled[labelled.SUBSET == 'raw']['label'].value_counts()

0          116
1           45
ABSTAIN      3
Name: label, dtype: int64

In [27]:
((tf.convert_to_tensor([1,2,3],dtype='float32')>1)).numpy()*1

array([0, 1, 1])

In [57]:

class ModelComparator:
    def __init__(self):
        
        self.class_weight = {0:1,1:3}
        self.sklearnModel = {'random':DummyClassifier(strategy ='stratified'),
                             'knn':KNeighborsClassifier(),
                             'baseMLP':MLPClassifier(512,max_iter=300),
                             'RF':RandomForestClassifier(class_weight = 'balanced',max_features=None),
                             'linSVC' : LinearSVC(class_weight=self.class_weight)}
        self.kerasModel = {'DocModel': DocModel,
                           'TokModel': TokModel,
                           'MixModel': MixModel}
        #self.kerasModel = {'MixModel1': (MixModel, 1e-4,0.995),
        #                   'MixModel2': (MixModel,5e-5, 0.995),
        #                   'MixModel3': (MixModel,5e-5, 0.99)}
        
    def setScorer(self,fracs=[]):
        self.score = {model:{'f1':{f:[] for f in fracs},'bal_acc':{f:[] for f in fracs}} 
                      for model in list(self.sklearnModel.keys()) + list(self.kerasModel.keys())}
        
        
        
    def findBestEpochs(self, df,epochs = [5,10,15], testSize = 30,nSplit = 5):
        self.epochScore = {model:{'f1':{f:[] for f in epochs},
                                  'bal_acc':{f:[] for f in epochs}} 
                           for model in  list(self.kerasModel.keys())}
        df = pd.concat([df[df.label == 1],df[df.label == 0]])
        lenDf = len(df)

        docEmbs,tokEmbs,label = (np.array(df[x].tolist()) for x in ['docEmbs','tokEmbs','label'])
        nTrain = lenDf-testSize
        sss = StratifiedShuffleSplit(n_splits = nSplit, 
                                     train_size = nTrain)
        
        for trainInd, testInd in  tqdm(sss.split(X = np.zeros(lenDf), y=df.label.tolist())):

            #test set for model evaluation
            docTest = tf.convert_to_tensor(docEmbs[testInd],dtype='float32')
            tokTest = tf.convert_to_tensor(tokEmbs[testInd],dtype='float32')
            labelTest = tf.convert_to_tensor(label[testInd],dtype='float32')
            
            #train set 
            docTrain = tf.convert_to_tensor(docEmbs[trainInd],dtype='float32')
            tokTrain = tf.convert_to_tensor(tokEmbs[trainInd],dtype='float32')
            labelTrain = tf.convert_to_tensor(label[trainInd],dtype='float32')
            
            for modelName in self.kerasModel:
                batch_size=8
                model = clone_model(self.kerasModel[modelName])
                last_epoch=0
                opt = Adam(learning_rate = self.kerasModel[modelName][1],epsilon=1e-9,beta_2 =self.kerasModel[modelName][2])
                model.compile(optimizer=opt,
                              loss =BinaryCrossentropy(),
                              weighted_metrics =[BinaryCrossentropy()])
                
                for epoch in epochs:
                    model.fit([tokTrain,docTrain],labelTrain,
                              batch_size=batch_size,
                              epochs=epoch,
                              initial_epoch = last_epoch,
                              class_weight=self.class_weight,
                              verbose = 2)

                    prediction = model.predict_on_batch([tokTest,docTest])
                    prediction = (prediction>0.5)*1

                    self.epochScore[modelName]['f1'][epoch].append(f1_score(labelTest,prediction))
                    self.epochScore[modelName]['bal_acc'][epoch].append(balanced_accuracy_score(labelTest,prediction))
                    last_epoch=epoch
        
    def trainAll(self,df,fracs=[],valSize = 0, testSize =30, nSplit = 10):
        if len(fracs)==0:
            fracs=[1]
        self.fracs = fracs
        self.setScorer(fracs)
        df = pd.concat([df[df.label == 1],df[df.label == 0]])
        lenDf = len(df)
        
        docEmbs = np.array(df.docEmbs.tolist())
        tokEmbs = np.array(df.tokEmbs.tolist())
        label = np.array(df.label.tolist())
            
        sss = StratifiedShuffleSplit(n_splits = nSplit, 
                                     test_size = testSize)
        
        for trainInd, testInd in  sss.split(X = np.zeros(lenDf), y=df.label.tolist()):

            #test set for model evaluation
            docTest = tf.convert_to_tensor(docEmbs[testInd],dtype='float32')
            tokTest = tf.convert_to_tensor(tokEmbs[testInd],dtype='float32')
            labelTest = label[testInd]
            
            # get validation set to monitor training
            if (valSize>0):
                valSplit = train_test_split(docEmbs[trainInd],tokEmbs[trainInd],
                                            label[trainInd],
                                            test_size = valSize,
                                            stratify = label[trainInd])
                (docTrainFull, docVal, 
                 tokTrainFull, tokVal,  
                 labelTrainFull, labelVal) = (tf.convert_to_tensor(vecs,dtype='float32') for vecs in valSplit)

                valSampleWeights = labelVal*self.class_weight[1] + (1-labelVal)*self.class_weight[0]
            else : 
                docTrainFull=docEmbs[trainInd]
                tokTrainFull=tokEmbs[trainInd]
                labelTrainFull=label[trainInd]
            
            for frac in fracs:
                # train given frac ratio
                if frac<1:
                    trainSplit = train_test_split(np.array(docTrainFull),np.array(tokTrainFull),
                                                  np.array(labelTrainFull),
                                                  train_size = int(frac*len(labelTrainFull)),
                                                  stratify = labelTrainFull)

                    (docTrain, _, 
                     tokTrain, _, 
                     labelTrain, _) = (tf.convert_to_tensor(vecs,dtype='float32') for vecs in trainSplit)
                else: 
                    (docTrain, tokTrain, labelTrain) = (docTrainFull,tokTrainFull,labelTrainFull)
                
                for modelName in self.sklearnModel:
                    model = clone(self.sklearnModel[modelName]).fit(docTrain,labelTrain)
                    prediction = model.predict(docTest)
                    
                    self.score[modelName]['f1'][frac].append(f1_score(labelTest,prediction))
                    self.score[modelName]['bal_acc'][frac].append(balanced_accuracy_score(labelTest,prediction))
                    del model
                for modelName in self.kerasModel:
                    model = clone_model(self.kerasModel[modelName])
                    epochs = 25
                    batch_size = 8
                    if modelName=='MixModelVal':
                        model.compile(optimizer=Adam(learning_rate = 1e-4,epsilon=1e-9,beta_2=0.995),
                                      loss =BinaryCrossentropy(),
                                      weighted_metrics =[BinaryCrossentropy()])
                        model.fit([tokTrain,docTrain],labelTrain,
                                  validation_data = ([tokVal,docVal],labelVal,valSampleWeights),
                                  batch_size=batch_size,
                                  validation_batch_size = valSize,
                                  epochs=epochs,
                                  class_weight=self.class_weight,
                                  verbose = 0,
                                  callbacks = [EarlyStopping(monitor = 'val_binary_crossentropy', 
                                                             mode = 'min',
                                                             patience=10, 
                                                             restore_best_weights=True,
                                                             start_from_epoch = 5,
                                                             verbose = 0)])
                    else:
                        model.compile(optimizer=Adam(learning_rate = 1e-4,epsilon=1e-9,beta_2=0.995),
                                      loss =BinaryCrossentropy(),
                                      weighted_metrics =[BinaryCrossentropy()])
                        model.fit([tokTrain,docTrain],labelTrain,
                                  batch_size=batch_size,
                                  epochs=epochs,
                                  initial_epoch = 10,
                                  class_weight=self.class_weight,
                                  verbose = 0)
                        
                    prediction = model.predict_on_batch([tokTest,docTest])
                    prediction = np.hstack((prediction>0.5)*1)
                    self.score[modelName]['f1'][frac].append(f1_score(labelTest,prediction))
                    self.score[modelName]['bal_acc'][frac].append(balanced_accuracy_score(labelTest,prediction))
                    del model
                    

    
    def plotScores(self):
        fig = make_subplots(rows=2, cols=1)
        for ind, scoreMetric in enumerate(['f1','bal_acc']): 
            for frac in self.fracs:
                fig.add_trace(go.Bar(
                    name=frac,
                    x=[model for model in self.score], 
                    y=[np.mean(self.score[model][scoreMetric][frac]) for model in self.score],
                    error_y=dict(type='data', 
                                 array =[np.std(self.score[model][scoreMetric][frac]) 
                                         for model in self.score])), row=ind+1, col=1)
            
        fig.update_layout(barmode='group')
        fig.show()
        
mc = ModelComparator()

In [62]:
# predict tout à 0 si trop d'unbalanced
score = mc.score
#mc.score = score

In [198]:
mc = ModelComparator()

In [60]:
mc.trainAll(labelled[labelled.SUBSET=='raw'],nSplit=6,fracs=[0.5,0.75,1])#[0.5,0.75,1])

/opt/anaconda/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning:

Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.

/opt/anaconda/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning:

Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.

/opt

/opt/anaconda/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning:

Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.



In [61]:
#import plotly.io as pio
#pio.renderers.default = 'iframe'


mc.plotScores()


In [189]:
mc.findBestEpochs(labelled[labelled.SUBSET=='raw'], nSplit= 3)

0it [00:00, ?it/s]

Epoch 1/5
17/17 - 15s - loss: 1.1524 - binary_crossentropy: 0.7519 - 15s/epoch - 902ms/step
Epoch 2/5
17/17 - 9s - loss: 1.0164 - binary_crossentropy: 0.7046 - 9s/epoch - 511ms/step
Epoch 3/5
17/17 - 9s - loss: 0.9585 - binary_crossentropy: 0.6121 - 9s/epoch - 508ms/step
Epoch 4/5
17/17 - 9s - loss: 0.8369 - binary_crossentropy: 0.5871 - 9s/epoch - 507ms/step
Epoch 5/5
17/17 - 9s - loss: 0.7447 - binary_crossentropy: 0.5061 - 9s/epoch - 509ms/step
Epoch 6/10
17/17 - 9s - loss: 0.5432 - binary_crossentropy: 0.3686 - 9s/epoch - 513ms/step
Epoch 7/10
17/17 - 9s - loss: 0.4049 - binary_crossentropy: 0.2578 - 9s/epoch - 510ms/step
Epoch 8/10
17/17 - 9s - loss: 0.2295 - binary_crossentropy: 0.1467 - 9s/epoch - 508ms/step
Epoch 9/10
17/17 - 8s - loss: 0.1621 - binary_crossentropy: 0.1085 - 8s/epoch - 473ms/step
Epoch 10/10
17/17 - 8s - loss: 0.0932 - binary_crossentropy: 0.0693 - 8s/epoch - 468ms/step
Epoch 11/15
17/17 - 8s - loss: 0.0370 - binary_crossentropy: 0.0245 - 8s/epoch - 483ms/step


1it [06:52, 412.83s/it]

Epoch 1/5
17/17 - 15s - loss: 1.1722 - binary_crossentropy: 0.7160 - 15s/epoch - 886ms/step
Epoch 2/5
17/17 - 8s - loss: 1.0176 - binary_crossentropy: 0.6743 - 8s/epoch - 498ms/step
Epoch 3/5
17/17 - 8s - loss: 1.0212 - binary_crossentropy: 0.6968 - 8s/epoch - 487ms/step
Epoch 4/5
17/17 - 8s - loss: 0.8933 - binary_crossentropy: 0.5859 - 8s/epoch - 487ms/step
Epoch 5/5
17/17 - 8s - loss: 0.8158 - binary_crossentropy: 0.5181 - 8s/epoch - 496ms/step
Epoch 6/10
17/17 - 9s - loss: 0.7277 - binary_crossentropy: 0.5289 - 9s/epoch - 505ms/step
Epoch 7/10
17/17 - 8s - loss: 0.4986 - binary_crossentropy: 0.3180 - 8s/epoch - 493ms/step
Epoch 8/10
17/17 - 9s - loss: 0.2454 - binary_crossentropy: 0.1804 - 9s/epoch - 501ms/step
Epoch 9/10
17/17 - 8s - loss: 0.1492 - binary_crossentropy: 0.0915 - 8s/epoch - 496ms/step
Epoch 10/10
17/17 - 8s - loss: 0.0502 - binary_crossentropy: 0.0324 - 8s/epoch - 496ms/step
Epoch 11/15
17/17 - 9s - loss: 0.0236 - binary_crossentropy: 0.0162 - 9s/epoch - 509ms/step


2it [13:27, 402.05s/it]

Epoch 1/5
17/17 - 14s - loss: 1.0985 - binary_crossentropy: 0.6983 - 14s/epoch - 803ms/step
Epoch 2/5
17/17 - 8s - loss: 1.0060 - binary_crossentropy: 0.7069 - 8s/epoch - 443ms/step
Epoch 3/5
17/17 - 8s - loss: 0.9277 - binary_crossentropy: 0.5515 - 8s/epoch - 453ms/step
Epoch 4/5
17/17 - 8s - loss: 0.8326 - binary_crossentropy: 0.5823 - 8s/epoch - 462ms/step
Epoch 5/5
17/17 - 8s - loss: 0.6759 - binary_crossentropy: 0.4666 - 8s/epoch - 462ms/step
Epoch 6/10
17/17 - 8s - loss: 0.4299 - binary_crossentropy: 0.2681 - 8s/epoch - 477ms/step
Epoch 7/10
17/17 - 8s - loss: 0.4073 - binary_crossentropy: 0.2824 - 8s/epoch - 472ms/step
Epoch 8/10
17/17 - 8s - loss: 0.2733 - binary_crossentropy: 0.1932 - 8s/epoch - 455ms/step
Epoch 9/10
17/17 - 8s - loss: 0.0999 - binary_crossentropy: 0.0694 - 8s/epoch - 464ms/step
Epoch 10/10
17/17 - 8s - loss: 0.0623 - binary_crossentropy: 0.0361 - 8s/epoch - 457ms/step
Epoch 11/15
17/17 - 8s - loss: 0.0282 - binary_crossentropy: 0.0170 - 8s/epoch - 465ms/step


3it [19:48, 396.08s/it]


In [ ]:
gc.collect()

In [191]:
fig = make_subplots(rows=2, cols=1)
for ind, scoreMetric in enumerate(['f1','bal_acc']): 
    for frac in [5,10,15]:
        fig.add_trace(go.Bar(
            name=frac,
            x=[model for model in mc.epochScore], 
            y=[np.mean(mc.epochScore[model][scoreMetric][frac]) for model in mc.epochScore],
            error_y=dict(type='data', 
                         array =[np.std(mc.epochScore[model][scoreMetric][frac]) 
                                 for model in mc.epochScore])), row=ind+1, col=1)

fig.update_layout(barmode='group')
fig.show()